<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [22]:
import pandas as pd
import warnings
import sklearn
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
import optuna
from tqdm import notebook 
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit 
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import mean_absolute_percentage_error

from tqdm import notebook
from lightgbm import log_evaluation, early_stopping

In [23]:
def delited(a):
    a=a.replace('-', '')
    a=a.replace(' ', '')
    return a

def cloud(a):
    a = str(a)
    a = a.replace('%', '')
    a = a.replace('.', '')
    if a == 'Облаков нет':
        return 0
    elif a =='Небо не видно из-за тумана и/или других метеорологических явлений':
        return 100
    elif '–' or ' ' in a:
        return a[:3]
    else:
        return a
    
def delited_1(a):
    a=str(a)
    a=a.replace('–', '')
    a=a.replace(' ', '')
    return a

def fill(a):
    if a == 'nan':
        return 100
    else:
        return a
    
def make_features(data, max_lag, rolling_mean_size):
    data['month'] = data.index.month
    data['day'] = data.index.day
    data['dayofweek'] = data.index.dayofweek
    data['is_weekend'] = data.dayofweek.isin([5,6])*1
    data['hour'] = data.index.hour
    
    for lag in range(1, max_lag+1):
        data['lag_{}'.format(lag)] = data['expenditure'].shift(lag)
    data['rolling_mean'] = data['expenditure'].shift(168).rolling(rolling_mean_size).mean()

In [24]:
factory = pd.read_csv('C://Users/Admin//Desktop//123//Самарэнерго/Предприятие.csv', sep=';')

C:\Users\Admin\AppData\Local\Temp\ipykernel_2184\2613400390.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  factory = pd.read_csv('C://Users/Admin//Desktop//123//Самарэнерго/Предприятие.csv', sep=';')


In [25]:
weather = pd.read_csv('C://Users//Admin//Desktop//123//Самарэнерго//Погодные_данные/weather_new.csv', sep=';', on_bad_lines='skip', decimal=",")

In [27]:
%%time
try:
    data_1 = factory.dropna(axis=1)
    data_1['datetime'] = pd.to_datetime(data_1['datetime'], format='%d.%m.%Y %H:%M')
    data_1['expenditure'] = data_1['expenditure'].apply(delited)
    data_1['expenditure'] = data_1['expenditure'].astype('int')
    data_1 = data_1.set_index('datetime')
    
    weather = weather[['Местное время в Самаре','T', 'U', 'Ff', 'N']] 
    weather = weather.rename(columns = {'Местное время в Самаре':'datetime'})
    weather['datetime'] = pd.to_datetime(weather['datetime'], format='%d.%m.%Y %H:%M')
    weather = weather.set_index('datetime')
    weather['N'] = weather['N'].apply(cloud).apply(delited_1).apply(fill).astype('int')
    weather = weather.fillna(method='bfill')
    weather = weather.resample('1H').median()
    weather = weather.reset_index()
    
    for i in range(len(weather)-2):
        if weather['T'][i]==0:
            continue
        elif weather['T'][i]==len(weather)-1:
            break
        elif weather['T'][i]!='NaN':
            continue
        elif weather['T'][i]=='NaN' and weather['T'][i+2]=='NaN':
            weather['T'][i]= weather['T'][i-1]
        elif weather['T'][i]=='NaN' and weather['T'][i+1] =='NaN':
            weather['T'][i]= (weather['T'][i-1]+weather['T'][i+2])/2
        elif weather['T'][i]=='NaN' and weather['T'][i+1]!='NaN':
            weather['T'][i]= (weather['T'][i-1] +weather['T'][i+1])/2
    weather = weather.set_index('datetime')
    weather = weather.fillna(method='bfill')
    
    data = data_1.join(weather, on='datetime')
    make_features(data, 7, 12)
    data = data.dropna()
    features = data.drop(['expenditure'], axis=1)
    target = data['expenditure'] 
    cat_features = ['month', 'day', 'dayofweek', 'is_weekend', 'hour']
    for i in notebook.tqdm(cat_features):
        features[i] = features[i].astype('str')
    features = pd.get_dummies(features, drop_first=True)
    numeric = ['T', 'U', 'N', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean']
    scaler = StandardScaler()
    features[numeric] = scaler.fit_transform(features[numeric])
    print('Подготовка данных завершена.')
    
except:
    print('Ошибка, проверьте вводимые даные.')
    
def objective_cv(trial, data=features, target=target):
    callbacks=[log_evaluation(period=0), early_stopping(stopping_rounds=200)]
    
    params = {
        'metric': 'rmse', 
        'random_state': 148,
        'n_estimators': 10000,
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.35,0.4,0.45,0.5,0.6,0.7,0.75,0.8,0.85]),
        'subsample': trial.suggest_categorical('subsample', [0.6,0.65,0.7,0.75,0.8,0.85]),
        'learning_rate': trial.suggest_categorical('learning_rate', 
                                                   [0.005,0.006,0.008,0.01,0.015,0.02,0.03]),
        'max_depth': trial.suggest_categorical('max_depth', [-1,10,20]),
        'num_leaves' : trial.suggest_int('num_leaves', 1, 300),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
        'verbosity': -1
    }
    cv = TimeSeriesSplit(n_splits=5)
    cv_scores = np.empty(5)
    for idx, (train_idx, test_idx) in enumerate(cv.split(data, target)):
        X_train, X_test = data.iloc[train_idx], data.iloc[test_idx]
        y_train, y_test = target.iloc[train_idx], target.iloc[test_idx]
        
        model = LGBMRegressor(**params)
        model.fit(X_train, y_train, eval_set=[(X_test,y_test)], callbacks=callbacks)
        preds = model.predict(X_test)
        cv_scores[idx] = mean_absolute_percentage_error(y_test, preds, )
        
    return np.mean(cv_scores)
print()
print('Обучение модели.')

study = optuna.create_study(direction="minimize", study_name="LGBM")
func = lambda trial: objective_cv(trial, features, target)
study.optimize(func, n_trials=5)

optuna_params = study.best_params

optuna_params['metric'] = 'mape'
optuna_params['random_state'] = 148
optuna_params['n_estimators'] = 10000

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.1, shuffle=False)
callbacks=[log_evaluation(period=300), early_stopping(stopping_rounds=200)]
model_optuna = LGBMRegressor(**optuna_params)
model_optuna.fit(X_train, y_train, eval_set=[(X_test, y_test)], callbacks=callbacks)


print()
print('Модель обучена и готова к работе.')


In [7]:
try:
    data_new = weather.join(data_1, on='datetime')
    make_features(data_new, 7, 12)
    scaler = StandardScaler()

    for i in notebook.tqdm(cat_features):
        data_new[i] = data_new[i].astype('str')
    data_new = pd.get_dummies(data_new, drop_first=True)
    data_new[numeric] = scaler.fit_transform(data_new[numeric])
    print()
    print('Введите, с какого дня начать прогноз, в формате "ГГГГ-ММ-ДД":')
    first_date = input()
    print()
    print('Введите, по какой день (включительно) сделать прогноз в формате "ГГГГ-ММ-ДД":')
    secord_date = input()
    
    time_for_predict = data_new[str(first_date+' 00:00:00'):str(secord_date+' 23:00:00')].index.to_list()
    
    for i in notebook.tqdm(time_for_predict):
        target_a = data_new.expenditure[i]
        features_a = data_new[str(i):str(i)].drop(['expenditure'], axis=1)
        prediction = model_optuna.predict(features_a)
        data_new.expenditure[str(i)] = (float(prediction))
        data_new = data_new[['T', 'U', 'Ff','N', 'expenditure']]
        make_features(data_new, 7, 12)
        for i in cat_features:
            data_new[i] = data_new[i].astype('str')
        data_new = pd.get_dummies(data_new, drop_first=True)
        data_new[numeric] = scaler.transform(data_new[numeric])
        
    prediction_april_test = data_new.expenditure[str(first_date+' 00:00:00'):]
    print('Предсказание завершено')
except:
    print()
    print('Ошибка. Проверьте, что данные ввода соответствует формату "ГГГГ-ММ-ДД"')


Введите, с какого дня начать прогноз, в формате "ГГГГ-ММ-ДД":
2022-06-28

Введите, по какой день (включительно) сделать прогноз в формате "ГГГГ-ММ-ДД":
2022-06-30


  0%|          | 0/72 [00:00<?, ?it/s]

Предсказание завершено


In [131]:
print('Куда сохранить файл? Введите адрес в формате',  r'C:\Users\User\Desktop\123\Самарэнерго\Предсказания\апрель1-3')
way_to_save = str(input())
prediction_april_test.to_csv(way_to_save) #и сохраняем
print()
print('Работа завершена. Данные сохранены в', way_to_save)

Куда сохранить файл? Введите адрес в формате C:\Users\User\Desktop\123\Самарэнерго\Предсказания\апрель1-3
C:\Users\User\Desktop\123\Самарэнерго\Предсказания\апрель1-3

Работа завершена. Данные сохранены в C:\Users\User\Desktop\123\Самарэнерго\Предсказания\апрель1-3
